<div dir="rtl" align="right">

# آشنایی با ایجنت‌ها (Agents) در LangChain
در این بخش، با مفهوم ایجنت‌ها در LangChain آشنا می‌شویم و یک سناریوی کاربردی را با هم پیاده‌سازی می‌کنیم.

ایجنت‌ها به مدل اجازه می‌دهند تا بر اساس نیاز، ابزارهای مختلفی مثل جستجو، ماشین حساب یا مدل زبانی را انتخاب و استفاده کند.

</div>

<div dir="rtl" align="right">

## سناریو: دستیار چندکاره
در این سناریو، یک دستیار هوشمند می‌سازیم که می‌تواند به سوالات کاربر پاسخ دهد؛ چه با جستجو در ویکی‌پدیا، چه با انجام محاسبات ریاضی، یا تولید متن خلاقانه.

مثال:
- "پایتخت فرانسه کجاست؟" (جستجو)
- "حاصل ۲۳ ضربدر ۴۵ چند می‌شود؟" (ماشین حساب)
- "یک شعر کوتاه درباره دریا بنویس." (مدل زبانی)

</div>

In [1]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
# leading the API key from environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
# choosing the language model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [4]:
# basic calculator tool
# this tool evaluates simple mathematical expressions
@tool
def calculator(expression: str) -> str:
    """محاسبه یک عبارت ریاضی ساده"""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"خطا: {e}"

In [5]:
# Wikipedia tool
# this tool queries Wikipedia for information
wiki_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [ ]:
tools = [calculator, wikipedia]

In [ ]:
# Agent prompt template
# this template defines how the agent should respond to user inputs
prompt = ChatPromptTemplate.from_messages([
    ("system", "شما یک دستیار هوشمند فارسی هستید که می‌توانید از ابزارهای مختلف برای پاسخ به سوالات استفاده کنید."),
    MessagesPlaceholder(variable_name="chat_history"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("human", "{input}")
])

# Creating the agent
# this agent uses the defined tools and prompt to answer questions
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# Running the agent with example questions
examples = [
    "پایتخت فرانسه کجاست؟",
    "حاصل ۲۳ ضربدر ۴۵ چند می‌شود؟",
    "یک شعر کوتاه درباره دریا بنویس."
]

# Displaying the results for each question
for question in examples:
    print(f"\n❓ سؤال: {question}")
    result = agent_executor.invoke({"input": question, "chat_history": []})
    print("✅ پاسخ:", result["output"])

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



❓ سؤال: پایتخت فرانسه کجاست؟

Invoking: `wikipedia` with `{'query': 'پایتخت فرانسه'}`


Page: Isfahan Seminary
Summary: The Isfahan Seminary (Persian: حوزه علمیه اصفهان) is one of the oldest seminaries in Isfahan, Iran. Currently, more than 40 schools in Isfahan province are under the supervision of the Management Center of Isfahan Seminary and the leadership of the supreme authority of Grand Ayatollah Hossein Mazaheri.

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


پایتخت فرانسه شهر پاریس است.

> Finished chain.
✅ پاسخ: پایتخت فرانسه شهر پاریس است.

❓ سؤال: حاصل ۲۳ ضربدر ۴۵ چند می‌شود؟

Invoking: `calculator` with `{'expression': '23*45'}`


1035

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


حاصل ضرب ۲۳ در ۴۵ برابر با ۱۰۳۵ می‌شود.

> Finished chain.
✅ پاسخ: حاصل ضرب ۲۳ در ۴۵ برابر با ۱۰۳۵ می‌شود.

❓ سؤال: یک شعر کوتاه درباره دریا بنویس.
دریا آرامشی بی پایان
در امواجش پنهان شده رازهای جهان

> Finished chain.
✅ پاسخ: دریا آرامشی بی پایان
در امواجش پنهان شده رازهای جهان


<div dir="rtl" align="right">

در این مثال، ایجنت به طور خودکار تشخیص می‌دهد که برای هر سوال باید از چه ابزاری استفاده کند و پاسخ مناسب را تولید می‌کند. شما می‌توانید سوالات خود را جایگزین کنید و عملکرد ایجنت را آزمایش کنید.

</div>